In [1]:
%pip install -r requirements.txt

  Using cached setuptools-75.2.0-py3-none-any.whl.metadata (6.9 kB)
Using cached setuptools-75.2.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%run -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from pathlib import Path

import pandas as pd
import spacy

f = Path(f"./tlg0525.tlg001.perseus-eng2.pickle")

def tokenize(df: pd.DataFrame):
    model = "en_core_web_sm"

    nlp = spacy.load(model, disable=["ner"])

    df["tokens"] = df["unannotated_strings"].apply(nlp.tokenizer)

    raw_texts = [t for t in df["unannotated_strings"]]
    annotated_texts = nlp.pipe(raw_texts, batch_size=100)

    df["nlp_docs"] = list(annotated_texts)

    return df

df = tokenize(pd.read_pickle(f))

# Bug 1

Why is the function `expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1)` returning `0.0` no matter what we pass to it?

In [5]:
from collections import Counter

def expected_frequency_of_collocation(df: pd.DataFrame, node: str, collocate: str, window_size: int = 1):
    """
    `node` and `collocate` should be the string representations
    of the associated lemmata
    """

    lemmata = [t.lemma_ for t in df['nlp_docs'].explode()]
    counter = Counter(lemmata)
    node_count = counter[node]
    collocate_count = counter[collocate]

    return (node_count * collocate_count * window_size) / len(lemmata)

expected_freq_the_mainlaind = expected_frequency_of_collocation(df, "mainland", "the")

expected_freq_the_mainlaind

3.2848751741271176

# Bug 1 analysis and response

Bug 1 didn't define t.lemma_ in the for loop that runs through the data frame. Lemmata then consists of simply tokens instead of lemmatized text. By modifying t to t.lemma_, we could fix the bug.

# Bug 2

Why does the code below not give us the number of occurrences of the token "the"?

In [50]:


node = "the"
observed_freq_the = df["tokens"].str.count(node)
observed_freq_the = df["tokens"].apply(lambda doc: sum(1 for token in doc if token.text == node))

observed_freq_the

0       13
1       10
2       22
3        4
4       13
        ..
3165     7
3166    16
3167     5
3168     3
3169     9
Name: tokens, Length: 3170, dtype: int64

# Bug 2 analysis and response

The str.count() function initially used looked for "the" in strings, but df["tokens"] isn't a string. df["tokens"], instead, is a sequence of tokens. So looking for strings directly won't work in a sequence of tokens.

# Bug 3

How is `o11` greater than `r1`, and why is our `o12` negative?

In [51]:
node = "mainland"
collocate = "the"

# def count_ngram_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
#     lemmata = [t.lemma_ for t in x]

#     # the right-hand side of a slice in Python is exclusive, so we add 1 to make sure
#     # we're actually getting one element to the right

#     chunked_lemmata = [lemmata[i - l_size:i + r_size + 1] for i in range(0, len(lemmata))]
    
#     cooccurrences = [1 for l in chunked_lemmata if w1 in l and w2 in l]

#     return sum(cooccurrences)

def count_ngram_collocations(x, w1, w2, l_size: int = 1, r_size: int = 1):
    lemmata = [t.lemma_ for t in x]

    indexes = [i for i, lemma in enumerate(lemmata) if lemma == w1]

    cooccurrences = 0

    for i in indexes:
        left = max(i - l_size, 0)
        right = min(i + r_size + 1, len(lemmata))
        window = lemmata[left:right]

        if w2 in window:
            cooccurrences += 1

    return cooccurrences

r1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == node])
r2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != node])
c1 = len([t for t in df["nlp_docs"].explode() if t.lemma_ == collocate])
c2 = len([t for t in df["nlp_docs"].explode() if t.lemma_ != collocate])

o11 = df["nlp_docs"].apply(count_ngram_collocations, args=(node, collocate)).sum()
o12 = r1 - o11
o21 = c1 - o11

print(f"r1: {r1}, r2: {r2}, c1: {c1}, c2: {c2}, o11: {o11}, o12: {o12}, o21: {o21}")

r1: 38, r2: 311516, c1: 26932, c2: 284622, o11: 23, o12: 15, o21: 26909


# Bug 3 analysis and response

The major problem is within the function count_ngram_collocations(). The chunked lemmata doesn't guaruntee that w1 is at the center, thus counting overlappingly and resulting in the frequency of the collocate pair being larger than the frequency of the node alone. There's some chance that w2 is counted as a collocate even if it's not in the same window with w1 but simply because the two windows overlap. By fixing the code, we ensure that w1 is always in the center of the window, and there are no more overlapping windows.